In [1]:
import os, sys
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
path_to_data = '/expanse/nfs/cw3e/cwp140/' 
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [3]:
## read the SNOTEL data
station_name = 'colorado_headwaters' ## upper_san_juan or colorado_headwaters
fname = path_to_data + 'downloads/SNOTEL/{0}.csv'.format(station_name)
df = pd.read_csv(fname)
# convert column names to a list
cols_list = df.columns.tolist()
# drop last 10 columns of summary information
df = df.drop(columns = cols_list[-10:])
## melt the dataframe to be a single column of values
df = pd.melt(df, id_vars=['date'], value_vars=cols_list[1:-10],var_name='year', value_name='value')
## remove rows where date doesn't make sense (e.g., Feb 29, 1987)
df = df.dropna(subset='value')
## set up time as index
df['time'] = pd.to_datetime(df['date'].values + df['year'], format='%m-%d%Y')
df = df.set_index('time')
df = df.drop(columns = ['date', 'year'])
## compute daily SWE accumulation
df['SWE_diff'] = df['value'].diff()
df

,value,SWE_diff
time,,
1986-10-01,0.368421,NaN
1986-10-02,0.268421,-0.100000
1986-10-03,0.215789,-0.052632
1986-10-04,0.163158,-0.052632
1986-10-05,0.131579,-0.031579
...,...,...
2023-09-26,0.000000,0.000000
2023-09-27,0.000000,0.000000
2023-09-28,0.000000,0.000000


In [4]:
## get 80th percentile value based on entire climatology 
## where SWE accumulation is at least 0.1 inch
idx = (df['SWE_diff'] >= 0.1)
tmp = df.loc[idx]
SWE_thres = tmp['SWE_diff'].describe(percentiles=[.80]).loc['80%'] # 80th percentile precipitation threshold
print(SWE_thres)

0.3675268816600003


In [5]:
## just get list of dates that exceed SWE 90th percentile threshold for WY 2022
idx = (df.index > '2022-10-01') & (df.index < '2023-09-30') & (df['SWE_diff'] >= SWE_thres)
tmp = df.loc[idx]
tmp

,value,SWE_diff
time,,
2022-10-27,1.032258,0.425806
2022-12-10,3.277419,0.696774
2022-12-24,5.003226,0.729032
2022-12-25,6.119355,1.116129
2022-12-27,6.677419,0.467742
2022-12-28,7.067742,0.390323
2022-12-29,7.438710,0.370968
2022-12-31,8.067742,0.406452
2023-01-01,8.416129,0.548387


In [6]:
tmp.to_csv('../data/SNOTEL_80th_perc_WY2023_{0}'.format(station_name))